In [31]:
# Imports
import pandas as pd
import numpy as np
import polars as pl
import os
import awpy
from demoparser2 import DemoParser
from awpy import Demo
from pathlib import Path
from awpy.stats import adr
from awpy.stats import kast
from awpy.stats import rating
from awpy.stats import calculate_trades
from supabase import create_client, Client
from dotenv import load_dotenv
from requests import post, get

In [32]:
folder_path = r'C:\Users\bayli\Documents\CS Demos\test_demos'
file_test = r'C:\Users\bayli\Documents\CS Demos\test_demos\natus-vincere-vs-faze-m2-mirage.dem'

# Creating DataFrames
df_flashes = pd.DataFrame()
df_he = pd.DataFrame()
df_infernos = pd.DataFrame()
df_smoke = pd.DataFrame()
df_kills = pd.DataFrame()
df_rounds = pd.DataFrame()
df_all_first_kills = pd.DataFrame()
df_adr = pd.DataFrame()
df_kast = pd.DataFrame()
df_util_dmg = pd.DataFrame()
team_rounds_won = pd.DataFrame()
players_id = pd.DataFrame()
df_matches = pd.DataFrame(columns=['event_id','match_name'])
i = 1
current_schema = "staging"
event_id = 3

In [33]:
def add_round_winners(ticks_df, rounds_df):
    ticks_df = ticks_df.to_pandas()
    rounds_df = rounds_df.to_pandas()

    # Makes sure the columns exists
    rounds_df['ct_team_clan_name'] = None
    rounds_df['t_team_clan_name'] = None
    rounds_df['winner_clan_name'] = None
    rounds_df['ct_team_current_equip_value'] = None
    rounds_df['t_team_current_equip_value'] = None
    rounds_df['ct_losing_streak'] = None
    rounds_df['t_losing_streak'] = None

    for idx, row in rounds_df.iterrows():
        freeze_end_tick = row['freeze_end']
        winner = row['winner']

        # Takes all corresponding entries
        first_tick_df = ticks_df[ticks_df['tick'] == freeze_end_tick]

        # Takes the name for every team
        try:
            CT_team = first_tick_df[first_tick_df['side'] == 'ct']['team_clan_name'].iloc[0]
        except IndexError:
            CT_team = None
        
        try:
            T_team = first_tick_df[first_tick_df['side'] == 't']['team_clan_name'].iloc[0]
        except IndexError:
            T_team = None

        # Takes the current equip value for every team
        try:
            CT_team_current_equip_value = first_tick_df[first_tick_df['side'] == 'ct']['current_equip_value'].sum()
        except KeyError:
            CT_team_current_equip_value = None

        try:
            T_team_current_equip_value = first_tick_df[first_tick_df['side'] == 't']['current_equip_value'].sum()
        except KeyError:
            T_team_current_equip_value = None

        # Determines the winner team name
        if winner == 'ct':
            winner_clan = CT_team
        elif winner in ['t', 'TERRORIST']:
            winner_clan = T_team
        else:
            winner_clan = None
            print(f"[!] Round {idx} - winner error: '{winner}'")
            
        # Fill Columns in the DataFrame
        rounds_df.at[idx, 'ct_team_clan_name'] = CT_team
        rounds_df.at[idx, 't_team_clan_name'] = T_team
        rounds_df.at[idx, 'winner_clan_name'] = winner_clan
        rounds_df.at[idx, 'ct_team_current_equip_value'] = CT_team_current_equip_value
        rounds_df.at[idx, 't_team_current_equip_value'] = T_team_current_equip_value


    return rounds_df

def add_losing_streaks(df: pd.DataFrame) -> pd.DataFrame:
    ct_losing_streak = []
    t_losing_streak = []

    ct_streak = 0
    t_streak = 0

    for _, row in df.iterrows():
        ct_team = row['ct_team_clan_name']
        t_team = row['t_team_clan_name']
        winner = row['winner_clan_name']
        
        if winner == ct_team:
            ct_streak = 0
            t_streak += 1
        else:  # winner == t_team
            t_streak = 0
            ct_streak += 1

        ct_losing_streak.append(ct_streak)
        t_losing_streak.append(t_streak)

    df['ct_losing_streak'] = ct_losing_streak
    df['t_losing_streak'] = t_losing_streak

    return df

def add_buy_type(row):

    if row['round_num'] in [1, 13]:
        return "Pistol", "Pistol"

    if row['ct_team_current_equip_value'] < 5000:
        ct_buy_type = "Full Eco"
    elif 5000 <= row['ct_team_current_equip_value'] < 10000:
        ct_buy_type = "Semi-Eco"
    elif 10000 <= row['ct_team_current_equip_value'] < 20000:
        ct_buy_type = "Semi-Buy"
    elif row['ct_team_current_equip_value'] >= 20000:
        ct_buy_type = "Full Buy"
    else:
        ct_buy_type = "Unknown"

    if row['t_team_current_equip_value'] < 5000:
        t_buy_type = "Full Eco"
    elif 5000 <= row['t_team_current_equip_value'] < 10000:
        t_buy_type = "Semi-Eco"
    elif 10000 <= row['t_team_current_equip_value'] < 20000:
        t_buy_type = "Semi-Buy"
    elif row['t_team_current_equip_value'] >= 20000:
        t_buy_type = "Full Buy"
    else:
        t_buy_type = "Unknown"

    return ct_buy_type, t_buy_type

def calculate_advantage_5v4(rounds_df, first_kills_df):

    # Makes sure the columns exists
    rounds_df['advantage_5v4'] = None

    # Checks what team got the first kill
    for idx, row in rounds_df.iterrows():
        round_num = row['round_num']

        # Filters the first kills DataFrame for the current round
        first_kill = first_kills_df[first_kills_df['round_num'] == round_num]

        if not first_kill.empty:
            # Gets the team that made the first kill
            killer_team = first_kill.iloc[0]['attacker_side']

            # Defines the advantage based on the killer team
            if killer_team == 'ct':
                rounds_df.at[idx, 'advantage_5v4'] = 'ct'
            elif killer_team == 't':
                rounds_df.at[idx, 'advantage_5v4'] = 't'

    return rounds_df

def insert_table(df, current_schema, table_name, conflict_cols):
    for row in df.to_dict(orient="records"):
        supabase.schema(current_schema).table(table_name).upsert(row, on_conflict=conflict_cols).execute()

def insert_or_update_player_history(players_df):
    for _, row in players_df.iterrows():
        steam_id = row["steam_id"]
        team_id = row["team_id"]
        
        # Finds out if the players have changed teams
        player_history_data = {
            "steam_id": steam_id,
            "team_id": team_id
        }
        # Updates the player history table with the new data
        supabase.table("player_history").upsert(
            player_history_data, 
            on_conflict=["steam_id", "team_id"]
        ).execute()

def rounds_correction(df: pl.DataFrame) -> pl.DataFrame:
    freeze_end_is_null = df.select(pl.col("freeze_end").first().is_null()).item()

    if freeze_end_is_null:
        df = df.with_columns(
            (pl.col("round_num") - 1).alias("round_num")
        )
        
    return df.filter(pl.col("freeze_end").is_not_null())

def fetch_all_rows(current_schema, table_name, page_size=1000):
    offset = 0
    all_data = []

    while True:
        response = supabase.schema(current_schema).table(table_name).select("*").range(offset, offset + page_size - 1).execute()
        data = response.data
        if not data:
            break
        all_data.extend(data)
        offset += page_size

    return all_data

In [ ]:
# Full Loop
for file_name in os.listdir(folder_path):
    if file_name.endswith('.dem'):

        file_path = os.path.join(folder_path, file_name)
        dem = Demo(file_path)
        dem.parse(player_props=["team_clan_name","total_rounds_played", "current_equip_value", "ct_losing_streak", "t_losing_streak"])

        # Gets all the Players' steam_ids
        this_file_players_id = dem.events.get('player_spawn')
        this_file_players_id = this_file_players_id.with_columns(
            this_file_players_id['user_steamid'].cast(pl.Utf8)
        )
        this_file_players_id = this_file_players_id.to_pandas()
        this_file_players_id = this_file_players_id[['user_steamid', 'user_name']].drop_duplicates()
        players_id = pd.concat([players_id, this_file_players_id], ignore_index=True)
        players_id = players_id[['user_steamid', 'user_name']].drop_duplicates()

        # Grenades Data
        # Makes that the data frame is not empty and that the columns are in the right format
        this_file_flashes = dem.events.get('flashbang_detonate', pl.DataFrame())
        if this_file_flashes is not None and len(this_file_flashes) > 0:
            this_file_flashes = this_file_flashes.with_columns(
                this_file_flashes['user_steamid'].cast(pl.Utf8)
            )
        this_file_he = dem.events.get('hegrenade_detonate', pl.DataFrame())
        if this_file_he is not None and len(this_file_he) > 0:
            this_file_he = this_file_he.with_columns(
                this_file_he['user_steamid'].cast(pl.Utf8)
            )
        this_file_infernos = dem.events.get('inferno_startburn', pl.DataFrame())
        if this_file_infernos is not None and len(this_file_infernos) > 0:
            this_file_infernos = this_file_infernos.with_columns(
                this_file_infernos['user_steamid'].cast(pl.Utf8)
            )
        this_file_smoke = dem.events.get('smokegrenade_detonate', pl.DataFrame())
        if this_file_smoke is not None and len(this_file_smoke) > 0:
            this_file_smoke = this_file_smoke.with_columns(
                this_file_smoke['user_steamid'].cast(pl.Utf8)
            )
        this_file_util_dmg = dem.events.get('player_hurt', pl.DataFrame())
        if this_file_util_dmg is not None and len(this_file_util_dmg) > 0:
            this_file_util_dmg = this_file_util_dmg.with_columns(
                this_file_util_dmg['attacker_steamid'].cast(pl.Utf8)
            )
        util_dmg = this_file_util_dmg.filter(
            (this_file_util_dmg["weapon"] == "hegrenade") |
            (this_file_util_dmg["weapon"] == "molotov")   |
            (this_file_util_dmg["weapon"] == "inferno")
        )
        # Makes sure that the data frames are not empty, converts them to pandas and appends them to the main data frame
        if this_file_flashes is not None and len(this_file_flashes) > 0:
            df_flashes = pd.concat([df_flashes, this_file_flashes.to_pandas()], ignore_index=True)
        if this_file_he is not None and len(this_file_he) > 0:
            df_he = pd.concat([df_he, this_file_he.to_pandas()], ignore_index=True)
        if this_file_infernos is not None and len(this_file_infernos) > 0:
            df_infernos = pd.concat([df_infernos, this_file_infernos.to_pandas()], ignore_index=True)
        if this_file_smoke is not None and len(this_file_smoke) > 0:
            df_smoke = pd.concat([df_smoke, this_file_smoke.to_pandas()], ignore_index=True)
        if this_file_util_dmg is not None and len(this_file_util_dmg) > 0:
            df_util_dmg = pd.concat([df_util_dmg, this_file_util_dmg.to_pandas()], ignore_index=True)

        # Opening Kills Data
        this_file_df_kills = awpy.stats.calculate_trades(demo=dem)
        this_file_df_kills = this_file_df_kills.with_columns(
            this_file_df_kills['attacker_steamid'].cast(pl.Utf8),
            this_file_df_kills['assister_steamid'].cast(pl.Utf8),
            this_file_df_kills['victim_steamid'].cast(pl.Utf8)
        )
        this_file_df_kills = this_file_df_kills.to_pandas()
        first_kills = this_file_df_kills.sort_values(by=['round_num', 'tick'])
        first_kills = first_kills.groupby('round_num').first().reset_index()
        df_all_first_kills = pd.concat([df_all_first_kills, first_kills], ignore_index=True)

        # Creates Match Table
        folder_name = os.path.basename(os.path.dirname(file_path))
        file_name = file_name.replace(f"{folder_name}_", "")
        df_matches = pd.concat([df_matches, pd.DataFrame({'match_name': [file_name], 'event_id': [event_id]})], ignore_index=True)

        # Rounds Data
        this_file_df_ticks = dem.ticks
        this_file_df_rounds = dem.rounds
        this_file_df_rounds = rounds_correction(this_file_df_rounds)
        this_file_df_rounds = add_round_winners(this_file_df_ticks,this_file_df_rounds)
        this_file_df_rounds = add_losing_streaks(this_file_df_rounds)
        this_file_df_rounds[['ct_buy_type', 't_buy_type']] = this_file_df_rounds.apply(add_buy_type, axis=1, result_type='expand')
        first_kills = this_file_df_kills.sort_values(by=['round_num', 'tick'])
        first_kills = first_kills.groupby('round_num').first().reset_index()
        df_all_first_kills = pd.concat([df_all_first_kills, first_kills], ignore_index=True)   

        this_file_df_rounds = calculate_advantage_5v4(this_file_df_rounds, df_all_first_kills)
        this_file_df_rounds['match_name'] = file_name
        df_rounds = pd.concat([df_rounds, this_file_df_rounds], ignore_index=True)
        df_rounds['event_id'] = event_id
        # Creates rounds won columns
        this_file_team_rounds_won = this_file_df_rounds.groupby('winner_clan_name').agg(
            total_rounds_won=('winner_clan_name', 'size'),
            t_rounds_won=('winner', lambda x: (x == 'ct').sum()),
            ct_rounds_won=('winner', lambda x: (x == 't').sum())
        ).reset_index()
        this_file_team_rounds_won.columns = ['team_clan_name', 'total_rounds_won','t_rounds_won', 'ct_rounds_won']
        team_rounds_won = pd.concat([team_rounds_won,this_file_team_rounds_won], ignore_index=True)
        df_kills = pd.concat([df_kills,this_file_df_kills], ignore_index=True)

        # ADR Data
        this_file_adr = awpy.stats.adr(demo=dem)
        this_file_adr = this_file_adr.with_columns(
            this_file_adr['steamid'].cast(pl.Utf8)
        )
        this_file_adr = this_file_adr.to_pandas()
        this_file_adr = this_file_adr.drop(['adr', 'name'], axis=1)
        df_adr = pd.concat([df_adr, this_file_adr], ignore_index=True)
        df_adr = df_adr.groupby(['steamid','side'], as_index=False).sum()
        df_adr = df_adr[df_adr['side'] != 'all']


        # KAST Data
        this_file_kast = awpy.stats.kast(demo=dem)
        this_file_kast = this_file_kast.with_columns(
            this_file_kast['steamid'].cast(pl.Utf8)
        )
        this_file_kast = this_file_kast.to_pandas()
        this_file_kast = this_file_kast.drop(['kast', 'name'], axis=1)
        df_kast = pd.concat([df_kast, this_file_kast], ignore_index=True)
        df_kast = df_kast.groupby(['steamid','side'], as_index=False).sum()
        df_kast = df_kast[df_kast['side'] != 'all']

        print(f"{i}: Processed {file_name}")
        i = i + 1

In [ ]:
dem = Demo(file_test)
dem.parse(player_props=["team_clan_name","total_rounds_played", "current_equip_value", "round_num"])
bomb = dem.events.get('bomb_planted')

In [41]:
bomb

ct_team_clan_name,ct_side,site,t_team_clan_name,t_side,tick,total_rounds_played,user_X,user_Y,user_Z,user_current_equip_value,user_health,user_place,user_name,user_steamid,user_team_clan_name,user_side,user_total_rounds_played
str,str,i32,str,str,i32,i32,f32,f32,f32,u32,i32,str,str,u64,str,str,i32
"""FaZe Clan""","""ct""",224,"""Natus Vincere""","""t""",29825,2,-344.895386,-1994.654541,-174.832825,4600,20,"""BombsiteA""","""Aleksib""",76561198013243326,"""Natus Vincere""","""t""",2
"""FaZe Clan""","""ct""",225,"""Natus Vincere""","""t""",35615,3,-1884.005615,242.032486,-159.96875,4900,100,"""BombsiteB""","""w0nderful""",76561199063068840,"""Natus Vincere""","""t""",3
"""FaZe Clan""","""ct""",224,"""Natus Vincere""","""t""",45507,4,-405.930908,-1986.646362,-179.902283,4300,100,"""BombsiteA""","""Aleksib""",76561198013243326,"""Natus Vincere""","""t""",4
"""FaZe Clan""","""ct""",225,"""Natus Vincere""","""t""",60000,6,-1992.987671,426.89975,-159.96875,4300,100,"""BombsiteB""","""iM""",76561198050250233,"""Natus Vincere""","""t""",6
"""FaZe Clan""","""ct""",225,"""Natus Vincere""","""t""",70671,7,-1987.031372,414.457611,-159.96875,4600,100,"""BombsiteB""","""b1t""",76561198246607476,"""Natus Vincere""","""t""",7
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Natus Vincere""","""ct""",224,"""FaZe Clan""","""t""",149954,13,-274.261108,-2156.402832,-173.633301,4900,100,"""BombsiteA""","""broky""",76561198201620490,"""FaZe Clan""","""t""",13
"""Natus Vincere""","""ct""",224,"""FaZe Clan""","""t""",156806,14,-527.68103,-2191.250244,-179.96875,4700,100,"""BombsiteA""","""broky""",76561198201620490,"""FaZe Clan""","""t""",14
"""Natus Vincere""","""ct""",225,"""FaZe Clan""","""t""",166297,15,-1987.031494,422.839417,-159.96875,4600,100,"""BombsiteB""","""broky""",76561198201620490,"""FaZe Clan""","""t""",15


In [43]:
# Rounds Data
this_file_df_ticks = dem.ticks
this_file_df_rounds = dem.rounds
this_file_df_rounds = rounds_correction(this_file_df_rounds)
this_file_df_rounds = add_round_winners(this_file_df_ticks,this_file_df_rounds)
this_file_df_rounds = add_losing_streaks(this_file_df_rounds)
this_file_df_rounds[['ct_buy_type', 't_buy_type']] = this_file_df_rounds.apply(add_buy_type, axis=1, result_type='expand')
this_file_df_rounds


,round_num,start,freeze_end,end,official_end,winner,reason,bomb_plant,bomb_site,ct_team_clan_name,t_team_clan_name,winner_clan_name,ct_team_current_equip_value,t_team_current_equip_value,ct_losing_streak,t_losing_streak,ct_buy_type,t_buy_type
0,1,529,14020,17002,17450,ct,t_killed,NaN,not_planted,FaZe Clan,Natus Vincere,FaZe Clan,4200,4050,0,1,Pistol,Pistol
1,2,17450,18730,21672,22120,ct,t_killed,NaN,not_planted,FaZe Clan,Natus Vincere,FaZe Clan,20650,1500,0,2,Full Buy,Full Eco
2,3,22120,23400,31167,31615,t,ct_killed,29825.0,bombsite_b,FaZe Clan,Natus Vincere,Natus Vincere,25200,21600,1,0,Full Buy,Full Buy
3,4,31615,32895,38239,38687,t,bomb_exploded,35615.0,bombsite_b,FaZe Clan,Natus Vincere,Natus Vincere,23300,23800,2,0,Full Buy,Full Buy
4,5,38687,39967,48131,48579,t,bomb_exploded,45507.0,bombsite_b,FaZe Clan,Natus Vincere,Natus Vincere,11050,25500,3,0,Semi-Buy,Full Buy
5,6,48579,49859,55334,55782,ct,t_killed,NaN,not_planted,FaZe Clan,Natus Vincere,FaZe Clan,27400,27350,0,1,Full Buy,Full Buy
6,7,55782,57062,60026,60474,t,ct_killed,60000.0,bombsite_b,FaZe Clan,Natus Vincere,Natus Vincere,30350,15800,1,0,Full Buy,Semi-Buy
7,8,60474,64802,73295,73743,t,bomb_exploded,70671.0,bombsite_b,FaZe Clan,Natus Vincere,Natus Vincere,6550,25850,2,0,Semi-Eco,Full Buy
8,9,73743,93834,100964,101412,ct,bomb_defused,99492.0,bombsite_b,FaZe Clan,Natus Vincere,FaZe Clan,26900,27650,0,1,Full Buy,Full Buy
9,10,101412,102692,107180,107628,t,bomb_exploded,104556.0,bombsite_b,FaZe Clan,Natus Vincere,Natus Vincere,25700,23500,1,0,Full Buy,Full Buy
